## Implementación de clustering para la selección de sucursales.


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [2]:
# Read excel file and create dataframe
df = pd.read_excel('DATOS PROYECTO LOGISTICA - Todos.xlsx', sheet_name='DATOS DE FUNDACIONES',  skiprows=5, usecols='B:L')
df.head()

,ORGANIZACIÓN,GRUPO POBLACIONAL,RED,LOCALIDAD,BARRIO,DIRECCIÓN,FAMILIAS ATENDIDAS,FRECUENCIA,SEMANA,Latitud,Longitud
0,Fundación 1,TERAPÉUTICO POR ADICCIÓN,ZIPAQUIRÁ,-,KM. 2 VIA SIMIJACA - VEREDA HATOCHICO,VDA HATO CHICO K M 2 BRR VIA SIMIJACA-CHIQUINQ...,20,MENSUAL,3 SEMANA,5.538380,-73.844520
1,Fundación 2,COMEDOR AMIGO,NACIONAL,ALBAN,VEREDA DE ALBAN CUNDINAMARCA SALON COMUNAL,VEREDA DE ALBAN CUNDINAMARCA SALON COMUNAL,150,SEMANAL,SEMANAL,4.930780,-74.450760
2,Fundación 3,ADULTO MAYOR,NACIONAL,ANOLAIMA,CENTRO,CARRERA 5 No. 127,65,SEMANAL,SEMANAL,4.763323,-74.463584
3,Fundación 4,TEJIENDO COMUNIDAD,NACIONAL,ANOLAIMA,VEREDA SAN JERÓNIMO DEL MUNICIPIO DE ANOLAIMA,VEREDA SAN JERÓNIMO DEL MUNICIPIO DE ANOLAIMA,66,MENSUAL,2 SEMANA,4.699704,-74.500822
4,Fundación 5,INTERNADO,SAN JOSÉ,ANTONIO NARIÑO,CIUDAD JARDÍN SUR,CRA. 12 B NO. 18 - 27 SUR CIUDAD JARDÍN SUR,120,MENSUAL,4 SEMANA,4.581430,-74.096940


# For seeing the distinct values of a column
```python
# Drop  DIRECCIÓN column
df.drop('DIRECCIÓN', axis=1, inplace=True)
# Print the value counts for each column
for col in df.columns:
    print(df[col].value_counts())
    print()
```


In [3]:
df['Latitud'] = df['Latitud'].astype(float)
# Drop nan values in latitud
df.dropna(subset=['Latitud'], inplace=True)
# print the column names and data types
print(df.dtypes)
# print the stats of the data
print(df.describe())

ORGANIZACIÓN           object
GRUPO POBLACIONAL      object
RED                    object
LOCALIDAD              object
BARRIO                 object
DIRECCIÓN              object
FAMILIAS ATENDIDAS      int64
FRECUENCIA             object
SEMANA                 object
Latitud               float64
Longitud              float64
dtype: object
       FAMILIAS ATENDIDAS       Latitud      Longitud
count          611.000000  6.110000e+02  6.110000e+02
mean            94.612111  7.503570e+04 -1.214464e+06
std            122.868309  1.854649e+06  3.001779e+07
min              5.000000  3.442037e+00 -7.419923e+08
25%             36.500000  4.574361e+00 -7.414648e+01
50%             60.000000  4.610360e+00 -7.410213e+01
75%            100.000000  4.672888e+00 -7.407669e+01
max           1500.000000  4.584399e+07  7.406401e+01


In [4]:
# If longitudes are positve, convert them to negative
df['Longitud'] = df['Longitud'].apply(lambda x: x * -1 if x > 0 else x)
# If longitudes are greater than 180 delete them
df = df[df['Longitud'] < 180]
# If longitudes are less than -180 delete them
df = df[df['Longitud'] > -180]
# If latitudes are greater than 90 delete them
df = df[df['Latitud'] < 90]
# If latitudes are less than -90 delete them
df = df[df['Latitud'] > -90]
print(df.shape)
# If longitudes are 1.2 standard deviations away from the mean, delete them
df = df[np.abs(df['Longitud'] - df['Longitud'].mean()) <= (2 * df['Longitud'].std())]
# If latitudes are 1.2 standard deviations away from the mean, delete them
df = df[np.abs(df['Latitud'] - df['Latitud'].mean()) <= (2 * df['Latitud'].std())]
print(df.shape)
print(df.describe())

(609, 11)
(579, 11)
       FAMILIAS ATENDIDAS     Latitud    Longitud
count          579.000000  579.000000  579.000000
mean            94.956822    4.620038  -74.112883
std            125.308974    0.066258    0.050565
min              5.000000    4.478338  -74.356028
25%             35.000000    4.574172  -74.144939
50%             60.000000    4.608428  -74.102165
75%            100.000000    4.667240  -74.077817
max           1500.000000    4.783006  -73.997710


In [5]:
# Print value conts for BARRIO
print(df['FAMILIAS ATENDIDAS'].value_counts())

50     78
100    46
20     39
60     38
30     37
       ..
554     1
151     1
56      1
145     1
125     1
Name: FAMILIAS ATENDIDAS, Length: 123, dtype: int64


In [6]:
# Using the plotly library plot a scatter plot of the data in the country of Colombia

# print the size of the dataframe

print(df.shape)

fig = px.scatter_mapbox(df,
                        lat="Latitud",
                        lon="Longitud",
                        color="BARRIO",
                        hover_name="ORGANIZACIÓN",
                        title="Fundaciones en Colombia",
                        zoom=5,

                        )
fig.update_layout(
                    mapbox_center_lat = 4.570868,
                    mapbox_center_lon = -74.297333,
                    mapbox_style="open-street-map",
                )
fig.show()
print('plot complete')

(579, 11)


plot complete


In [7]:
# K-means clustering
# Create a copy of the dataframe
df.dropna(inplace=True)
df_copy = df.copy()


print(df_copy.columns)
# Use one hot encoding to convert categorical data to numerical data

# Create a list of the categorical columns

categorical_columns = ['GRUPO POBLACIONAL', 'BARRIO', 'RED', 'FRECUENCIA', 'SEMANA']

# Create a dataframe with the one hot encoded data
df_one_hot = pd.get_dummies(df_copy, columns=categorical_columns)

# Print the shape of the dataframe

print(df_one_hot.shape)


Index(['ORGANIZACIÓN', 'GRUPO POBLACIONAL', 'RED', 'LOCALIDAD', 'BARRIO',
       'DIRECCIÓN', 'FAMILIAS ATENDIDAS', 'FRECUENCIA', 'SEMANA', 'Latitud',
       'Longitud'],
      dtype='object')
(576, 478)


In [8]:
scaler = StandardScaler()

df_one_hot_scaled = scaler.fit_transform(df_one_hot[[c for c in df_one_hot.columns if c != 'ORGANIZACIÓN' and c != 'DIRECCIÓN' and c != 'LOCALIDAD']])


kmeans= KMeans(n_clusters=6, random_state=0)
df['kmeans'] = kmeans.fit_predict(df_one_hot_scaled)


In [9]:
# Get the null values
df.isnull().sum()


ORGANIZACIÓN          0
GRUPO POBLACIONAL     0
RED                   0
LOCALIDAD             0
BARRIO                0
DIRECCIÓN             0
FAMILIAS ATENDIDAS    0
FRECUENCIA            0
SEMANA                0
Latitud               0
Longitud              0
kmeans                0
dtype: int64

In [10]:
df.head()

,ORGANIZACIÓN,GRUPO POBLACIONAL,RED,LOCALIDAD,BARRIO,DIRECCIÓN,FAMILIAS ATENDIDAS,FRECUENCIA,SEMANA,Latitud,Longitud,kmeans
4,Fundación 5,INTERNADO,SAN JOSÉ,ANTONIO NARIÑO,CIUDAD JARDÍN SUR,CRA. 12 B NO. 18 - 27 SUR CIUDAD JARDÍN SUR,120,MENSUAL,4 SEMANA,4.581430,-74.096940,4
5,Fundación 6,TERAPÉUTICO POR ADICCIÓN,INMACULADA CONCEPCIÓN,ANTONIO NARIÑO,LA ESTANZUELA,DIAG. 44 NO. 17 - 40,554,QUINCENAL,2 Y 4 SEMANA DEL MES,4.601168,-74.087721,1
6,Fundación 7,TEJIENDO COMUNIDAD,INMACULADA CONCEPCIÓN,ANTONIO NARIÑO,EDUARDO SANTOS,CLL 1C BIS N.19-18,40,QUINCENAL,1 Y 3 SEMANA DEL MES,4.594741,-74.091919,1
7,Fundación 8,TERAPÉUTICO POR DISCAPACIDAD,SAN JOSÉ,ANTONIO NARIÑO,CIUDAD BERNA,AV CARACAS NO. 6-11 SUR\nCALLE 13 A SUR No. 12...,107,QUINCENAL,1 Y 3 SEMANA DEL MES,4.583549,-74.095351,4
8,Fundación 9,TEJIENDO COMUNIDAD,ESPÍRITU SANTO,ANTONIO NARIÑO,LA F RAGUA,CALLE 14 A SUR # 28 - 46,50,QUINCENAL,2 Y 4 SEMANA DEL MES,4.595307,-74.105222,1


In [11]:
fig = px.scatter_mapbox(df,
                        lat="Latitud",
                        lon="Longitud",
                        color="kmeans",
                        hover_name="ORGANIZACIÓN",
                        title="Fundaciones en Colombia",
                        zoom=5,

                        )
fig.update_layout(
                    mapbox_center_lat = 4.570868,
                    mapbox_center_lon = -74.297333,
                    mapbox_style="open-street-map",
                )
fig.show()
print('plot complete')

plot complete


In [12]:
# Create a new dataframe with only the row with kmeans = 1
df_kmeans_1 = df[df['kmeans'] == 1]
# drop the kmeans column
df_kmeans_1.drop(columns=['kmeans'], inplace=True)
print(df_kmeans_1.shape)
df_kmeans_1.head()

(325, 11)


/tmp/ipykernel_5398/4098077425.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ORGANIZACIÓN,GRUPO POBLACIONAL,RED,LOCALIDAD,BARRIO,DIRECCIÓN,FAMILIAS ATENDIDAS,FRECUENCIA,SEMANA,Latitud,Longitud
5,Fundación 6,TERAPÉUTICO POR ADICCIÓN,INMACULADA CONCEPCIÓN,ANTONIO NARIÑO,LA ESTANZUELA,DIAG. 44 NO. 17 - 40,554,QUINCENAL,2 Y 4 SEMANA DEL MES,4.601168,-74.087721
6,Fundación 7,TEJIENDO COMUNIDAD,INMACULADA CONCEPCIÓN,ANTONIO NARIÑO,EDUARDO SANTOS,CLL 1C BIS N.19-18,40,QUINCENAL,1 Y 3 SEMANA DEL MES,4.594741,-74.091919
8,Fundación 9,TEJIENDO COMUNIDAD,ESPÍRITU SANTO,ANTONIO NARIÑO,LA F RAGUA,CALLE 14 A SUR # 28 - 46,50,QUINCENAL,2 Y 4 SEMANA DEL MES,4.595307,-74.105222
9,Fundación 10,TERAPÉUTICO POR DISCAPACIDAD,INMACULADA CONCEPCIÓN,ANTONIO NARIÑO,LA ESTANZUELA,DIAGONAL 4 A NO. 18A-08,30,QUINCENAL,2 Y 4 SEMANA DEL MES,4.598645,-74.089669
10,Fundación 11,TEJIENDO COMUNIDAD,SAN JOSÉ,ANTONIO NARIÑO,CUIDAD BERNA,CRA. 11 # 7 - 07 SUR,30,QUINCENAL,1 Y 3 SEMANA DEL MES,4.599920,-74.079962


## Hallar la matriz de distancias entre las sucursales

In [13]:
organizacion_name = "BANCO DE ALIMENTOS DE BOGOTA"
banco_de_alimentos_coordenadas = [4.620809028221943, -74.08970640057449]

# Add banco de alimentos 
# Create a new dataframe with only the columns latitud, longitud, organizacion
df_distancias = df_kmeans_1[['Latitud', 'Longitud', 'ORGANIZACIÓN']]
# Add a new row with the coordenadas of the banco de alimentos
df_distancias = df_distancias.append({'Latitud': banco_de_alimentos_coordenadas[0], 'Longitud': banco_de_alimentos_coordenadas[1], 'ORGANIZACIÓN': organizacion_name}, ignore_index=True)

df_distancias.tail()

/tmp/ipykernel_5398/273733597.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Latitud,Longitud,ORGANIZACIÓN
321,4.748265,-74.044313,Fundación 620
322,4.675057,-74.021175,Fundación 621
323,4.601318,-74.069278,Fundación 622
324,4.622414,-74.162582,Fundación 623
325,4.620809,-74.089706,BANCO DE ALIMENTOS DE BOGOTA


In [17]:
# Create a dictionary to store the distances
from haversine import haversine, Unit
distancias = {}
# Iterate over the rows of the dataframe
for index, row in df_distancias.iterrows():
    for index2, row2 in df_distancias.iterrows():
        # Calculate the distance between the two rows using the haversine formula
        distancia = haversine((row['Latitud'], row['Longitud']), (row2['Latitud'], row2['Longitud']))

        # Add the distance to the dictionary

        distancias[(row['ORGANIZACIÓN'], row2['ORGANIZACIÓN'])] = distancia

In [21]:
# Create a csv file with the distances
import csv
with open('distancias.csv', 'w') as f:
    for key in distancias.keys():
        f.write(f'{key[0]},{key[1]},{distancias[key]}\n')
